In [93]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sqlalchemy
import h5py

In [94]:
db_connection_string = 'sqlite:///./Resources/product.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['AAPL_1_Day_Candles', 'AAPL_1_Min_Candles', 'AAPL_Indicators', 'AAPL_Info', 'MSFT_1_Day_Candles', 'MSFT_1_Min_Candles', 'MSFT_Indicators', 'MSFT_Info']


In [95]:
ticker = 'MSFT'
indicators_df = pd.read_sql_table(ticker + '_Indicators', con=engine, index_col='Datetime')

---

## Prepare the data to be used on a neural network model

In [96]:
print(indicators_df.head())

                       Open    High     Low   Close  Volume  CDL2CROWS  \
Datetime                                                                 
2021-09-07 06:42:00  300.71  300.71  300.61  300.61    1352          0   
2021-09-07 06:54:00  300.72  300.72  300.70  300.70    1376          0   
2021-09-07 06:56:00  300.70  300.70  300.70  300.70     512          0   
2021-09-07 06:58:00  300.70  300.70  300.70  300.70    1195          0   
2021-09-07 06:59:00  300.65  300.65  300.65  300.65     460          0   

                     CDL3BLACKCROWS  CDL3INSIDE  CDL3LINESTRIKE  CDL3OUTSIDE  \
Datetime                                                                       
2021-09-07 06:42:00               0           0               0            0   
2021-09-07 06:54:00               0           0               0            0   
2021-09-07 06:56:00               0           0               0            0   
2021-09-07 06:58:00               0           0               0            0   
2

In [97]:
# Review the data types associated with the columns
indicators_df.dtypes

Open                float64
High                float64
Low                 float64
Close               float64
Volume                int64
CDL2CROWS             int64
CDL3BLACKCROWS        int64
CDL3INSIDE            int64
CDL3LINESTRIKE        int64
CDL3OUTSIDE           int64
CDL3STARSINSOUTH      int64
Trade Signal        float64
Upper Band          float64
Middle Band         float64
Lower Band          float64
DEMA                float64
EMA                 float64
HT_TRENDLINE        float64
KAMA                float64
dtype: object

In [98]:
display(indicators_df['Trade Signal'].value_counts())

 0.0    10225
-1.0      251
 1.0      211
Name: Trade Signal, dtype: int64

### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [99]:
# Create a list of categorical variables 
categorical_variables = ['Trade Signal']

# Display the categorical variables list
display(categorical_variables)

['Trade Signal']

In [100]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [101]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(indicators_df[categorical_variables])


In [102]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)
# Review the DataFrame
display(encoded_df.head(10))

,Trade Signal_-1.0,Trade Signal_0.0,Trade Signal_1.0
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
5,0.0,1.0,0.0
6,0.0,1.0,0.0
7,0.0,1.0,0.0
8,0.0,1.0,0.0
9,0.0,1.0,0.0


In [103]:
encoded_df.rename(columns={'Trade Signal_-1.0': 'Bearish', 'Trade Signal_0.0': 'None', 'Trade Signal_1.0':'Bullsih'}, inplace=True)
encoded_df.drop(columns='None', inplace=True)
print(encoded_df.head())

   Bearish  Bullsih
0      0.0      0.0
1      0.0      0.0
2      0.0      0.0
3      0.0      0.0
4      0.0      0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [104]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df.copy()

# Display a sample of y
display(y.head())

,Bearish,Bullsih
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [105]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = indicators_df.drop(columns=['Trade Signal'])

# Review the features DataFrame
display(X.head())

,Open,High,Low,Close,Volume,CDL2CROWS,CDL3BLACKCROWS,CDL3INSIDE,CDL3LINESTRIKE,CDL3OUTSIDE,CDL3STARSINSOUTH,Upper Band,Middle Band,Lower Band,DEMA,EMA,HT_TRENDLINE,KAMA
Datetime,,,,,,,,,,,,,,,,,,
2021-09-07 06:42:00,300.71,300.71,300.61,300.61,1352,0,0,0,0,0,0,300.899851,300.632,300.364149,300.505521,300.643410,300.608905,300.611546
2021-09-07 06:54:00,300.72,300.72,300.70,300.70,1376,0,0,0,0,0,0,300.840200,300.690,300.539800,300.521484,300.647061,300.611291,300.612119
2021-09-07 06:56:00,300.70,300.70,300.70,300.70,512,0,0,0,0,0,0,300.763833,300.668,300.572167,300.536196,300.650477,300.614092,300.612950
2021-09-07 06:58:00,300.70,300.70,300.70,300.70,1195,0,0,0,0,0,0,300.763563,300.686,300.608437,300.549753,300.653672,300.617068,300.613833
2021-09-07 06:59:00,300.65,300.65,300.65,300.65,460,0,0,0,0,0,0,300.745103,300.672,300.598897,300.555999,300.653435,300.619475,300.614086


### Step 6: Split the features and target sets into training and testing datasets.


In [106]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [107]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [108]:
# Define the the number of inputs (features) to the model
number_input_features = len(list(X.columns))
# Review the number of features
number_input_features


18

In [109]:
# Define the number of neurons in the output layer
number_output_neurons = 2

In [110]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  int(round((number_input_features + number_output_neurons)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1


10

In [111]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  int(round((hidden_nodes_layer1 + number_output_neurons)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


6

In [112]:
# Create the Sequential model instance
nn = Sequential()


In [113]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))


In [114]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [115]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [116]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                190       
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 14        
Total params: 270
Trainable params: 270
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [117]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [118]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [119]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

84/84 [==============================] - 0s 373us/step - loss: 6.5129e-07 - accuracy: 0.2055
Loss: 6.512850632134359e-07, Accuracy: 0.20546406507492065


### Step 4: Save and export your model to an HDF5 file

In [120]:
# Set the model's file path
filename = "DLNN_model_1"
file_path = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn.save(file_path)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1 - Deep learning model with 2 hidden layers

In [121]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


18


In [122]:
# Review the number of features
number_input_features

18

In [123]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 2

In [124]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 =  int(round((number_input_features + number_output_neurons_A1)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A1


10

In [125]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  int(round((hidden_nodes_layer1_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A1


6

In [126]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A1 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A1


4

In [127]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [128]:
# Add the first hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation="relu", input_dim=number_input_features))


In [129]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))


In [130]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer3_A1, activation="relu"))


In [131]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))


In [132]:
# Display the Sequential model summary
nn_A1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 10)                190       
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 28        
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 10        
Total params: 294
Trainable params: 294
Non-trainable params: 0
_________________________________________________________________


In [133]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [134]:
# Fit the model using 50 epochs and the training data
model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Alternative Model 2 - Deep learning model with 3 hidden layers

In [135]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


18


In [136]:
# Review the number of features
number_input_features

18

In [137]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 2

In [138]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 =  int(round((number_input_features + number_output_neurons_A2)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A2


10

In [139]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  int(round((hidden_nodes_layer1_A2 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2


6

In [140]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A2 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A2


4

In [141]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer4_A2 =  int(round((hidden_nodes_layer3_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer4_A2


3

In [142]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [143]:
# Add the first hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu", input_dim=number_input_features))


In [144]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))


In [145]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu"))


In [146]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2, activation="relu"))


In [147]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))


In [148]:
# Display the Sequential model summary
nn_A2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 10)                190       
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_25 (Dense)             (None, 4)                 28        
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 15        
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 8         
Total params: 307
Trainable params: 307
Non-trainable params: 0
_________________________________________________________________


In [149]:
# Compile the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [150]:
# Fit the model using 50 epochs and the training data
model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50, verbose=0)


#### Alternative Model 3 - Principal Component Analysis

In [151]:
from sklearn.decomposition import PCA


In [152]:
# Create a StandardScaler instance
pca_scaler = StandardScaler()

# Fit the scaler to the features
X_pca_scaler = pca_scaler.fit(X)

# Fit the scaler to the features
X_pca_scaled = X_pca_scaler.transform(X)


In [179]:
# for n in range(2, number_input_features):
n = number_input_features
best_n = n
total_exp_var = 100.0
while total_exp_var > 99.999:
# for n in range(number_input_features, 2, -1):
    pca = PCA(n_components=n, random_state=1)
    X_pca = pca.fit_transform(X_pca_scaled)
    exp_var = pca.explained_variance_ratio_
#     print(f'{exp_var}')
    total_exp_var = 100*exp_var.sum()
    print(f'N = {n:3}: Total explained variance: {total_exp_var:.4f}%')

    if total_exp_var > 99.999:
        best_n = n
    n -= 1

N =  18: Total explained variance: 100.0000%
[6.86794810e-01 6.29082829e-02 6.25265665e-02 6.24989296e-02
 6.24708183e-02 6.20470866e-02 4.67382148e-04 1.19356004e-04
 6.15629832e-05 4.47669701e-05 2.50777523e-05 1.38510625e-05
 1.27873698e-05 6.84584658e-06 1.87569912e-06 1.14566254e-32
 3.22164849e-34 1.78511298e-38] 18 18
N =  17: Total explained variance: 100.0000%
[6.86794810e-01 6.29082829e-02 6.25265665e-02 6.24989296e-02
 6.24708183e-02 6.20470866e-02 4.67382148e-04 1.19356004e-04
 6.15629832e-05 4.47669701e-05 2.50777523e-05 1.38510625e-05
 1.27873698e-05 6.84584658e-06 1.87569912e-06 1.14566254e-32
 3.22164849e-34] 17 18
N =  16: Total explained variance: 100.0000%
[6.86794810e-01 6.29082829e-02 6.25265665e-02 6.24989296e-02
 6.24708183e-02 6.20470866e-02 4.67382148e-04 1.19356004e-04
 6.15629832e-05 4.47669701e-05 2.50777523e-05 1.38510625e-05
 1.27873698e-05 6.84584658e-06 1.87569912e-06 1.14566254e-32] 16 17
N =  15: Total explained variance: 100.0000%
[6.86794810e-01 6.29

In [181]:
print(best_n)
pca = PCA(n_components=best_n, random_state=1)
X_pca = pca.fit_transform(X_pca_scaled)
exp_var = pca.explained_variance_ratio_
print(f'N = {best_n:3}: Total explained variance: {100*exp_var.sum():.4f}%')


13
N =  13: Total explained variance: 99.9991%


In [182]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, random_state=1)

In [183]:
# Define the the number of inputs (features) to the model
number_input_features = best_n
print(number_input_features)


13


In [184]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 2

In [185]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 =  int(round((number_input_features + number_output_neurons_A3)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A3


8

In [186]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 =  int(round((hidden_nodes_layer1_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A3


5

In [187]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A3 =  int(round((hidden_nodes_layer2_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the third layer
hidden_nodes_layer3_A3


4

In [188]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [189]:
# Add the first hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, activation="relu", input_dim=number_input_features))


In [190]:
# Add the second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))


In [191]:
# Add the third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))


In [192]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A3.add(Dense(units=number_output_neurons_A3, activation="sigmoid"))


In [193]:
# Display the Sequential model summary
nn_A3.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_33 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 24        
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 10        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [194]:
# Compile the Sequential model
nn_A3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [195]:
# Fit the model using 50 epochs and the training data
model_A3 = nn_A3.fit(X_pca_train, y_pca_train, epochs=50, verbose=1)


Epoch 1/50
251/251 [==============================] - 0s 492us/step - loss: 0.5812 - accuracy: 0.6519
Epoch 2/50
251/251 [==============================] - 0s 490us/step - loss: 0.1209 - accuracy: 0.9988
Epoch 3/50
251/251 [==============================] - 0s 483us/step - loss: 0.0568 - accuracy: 0.9990
Epoch 4/50
251/251 [==============================] - 0s 471us/step - loss: 0.0407 - accuracy: 0.9994
Epoch 5/50
251/251 [==============================] - 0s 475us/step - loss: 0.0307 - accuracy: 0.9994
Epoch 6/50
251/251 [==============================] - 0s 479us/step - loss: 0.0221 - accuracy: 0.9994
Epoch 7/50
251/251 [==============================] - 0s 479us/step - loss: 0.0171 - accuracy: 0.9994
Epoch 8/50
251/251 [==============================] - 0s 479us/step - loss: 0.0141 - accuracy: 0.9994
Epoch 9/50
251/251 [==============================] - 0s 479us/step - loss: 0.0113 - accuracy: 0.9999
Epoch 10/50
251/251 [==============================] - 0s 483us/step - loss: 0.007

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [196]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
84/84 [==============================] - 0s 420us/step - loss: 6.5129e-07 - accuracy: 0.2055
Loss: 6.512850632134359e-07, Accuracy: 0.20546406507492065


In [197]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
84/84 [==============================] - 0s 421us/step - loss: 7.7542e-07 - accuracy: 1.0000
Loss: 7.754246098556905e-07, Accuracy: 1.0


In [198]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
84/84 [==============================] - 0s 433us/step - loss: 5.1365e-05 - accuracy: 0.0449
Loss: 5.1365284889470786e-05, Accuracy: 0.04491018131375313


In [199]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_pca_test, y_pca_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
84/84 [==============================] - 0s 409us/step - loss: 3.6888e-07 - accuracy: 0.9263
Loss: 3.6888212662233855e-07, Accuracy: 0.926272451877594


### Run different numbers of epochs on best model

In [200]:
# Fit the model using the training data with different numbers of epochs
model_list = []
for n in range(50, 151, 20):
    print(f'Fitting model, epochs: {n}')
    model_list.append(nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False))
#     model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False)

    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
    model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

Fitting model, epochs: 50
84/84 [==============================] - 0s 409us/step - loss: 1.7584e-07 - accuracy: 1.0000
Loss: 1.7584271461146272e-07, Accuracy: 1.0
Fitting model, epochs: 70
84/84 [==============================] - 0s 409us/step - loss: 4.5019e-08 - accuracy: 1.0000
Loss: 4.501921679889165e-08, Accuracy: 1.0
Fitting model, epochs: 90
84/84 [==============================] - 0s 421us/step - loss: 1.4962e-08 - accuracy: 1.0000
Loss: 1.4961877070618357e-08, Accuracy: 1.0
Fitting model, epochs: 110
84/84 [==============================] - 0s 426us/step - loss: 7.0936e-09 - accuracy: 1.0000
Loss: 7.0935604057353885e-09, Accuracy: 1.0
Fitting model, epochs: 130
84/84 [==============================] - 0s 397us/step - loss: 4.3638e-09 - accuracy: 1.0000
Loss: 4.36381464297142e-09, Accuracy: 1.0
Fitting model, epochs: 150
84/84 [==============================] - 0s 399us/step - loss: 3.1043e-09 - accuracy: 1.0000
Loss: 3.104312806812004e-09, Accuracy: 1.0


In [201]:
# for model in model_list:
#     pass
#     print(model.history)
    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
#     model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
#     print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

### Step 3: Save each of your alternative models as an HDF5 file.


In [202]:
# Set the file path for the first alternative model
filename = "DLNN_model_A1"
file_path_A1 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A1.save(file_path_A1)


In [203]:
# Set the file path for the second alternative model
filename = "DLNN_model_A2"
file_path_A2 = Path('Resources/' + filename + '.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path_A2)


In [204]:
# # Set the file path for the third alternative model
filename = "DLNN_model_A3"
file_path_A3 = Path('Resources/' + filename + '.h5')

# # Export your model to a HDF5 file
nn_A3.save(file_path_A3)
